In [ ]:
# Installing dependencies
# Installing Gradio
%pip install gradio
%pip install fsspec --upgrade
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.5 MB/s et

In [ ]:
%load_ext gradio

# Recommendation Function

In [ ]:
# GitHub
!git clone https://github.com/Hberto/Move-in-Assistance.git
%cd Move-in-Assistance
!git pull

In [ ]:

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

#def make_recommandation(in_miet, in_gruen, in_aqi):

def make_recommandation(in_infra_rad, in_miet, in_gruen, in_aqi, in_rad_house, in_flach):
  infra_rad = float(in_infra_rad)
  miet = float(in_miet)
  gruen = float(in_gruen)
  aqi = float(in_aqi)
  rad_house = float(in_rad_house)
  flach = float(in_flach)

  data = pd.read_csv('files/out/merged_featureset_nonan.csv')
  # User input (replace with actual user input)
  user_input = {'Anzahl_Punkte': infra_rad, 'Mietpreis pro qm': miet, 'Gruenflaeche': gruen, 'AQI': aqi, 'Fahrradhausanzahl': rad_house, 'Flaeche': flach}
  user_input_features = list(user_input.keys())
  user_input_values = [user_input[feature] for feature in user_input_features]
  user_input_array = [user_input_values]

  selected_data = data[user_input_features].values

  similarity_scores = cosine_similarity(user_input_array, selected_data)

  top_districts_indices = similarity_scores.argsort()[0][-3:][::-1]
  top_districts = data.loc[top_districts_indices, 'Stadtteil']

  return list(top_districts)


In [ ]:
%reload_ext gradio
import gradio as gr

with gr.Blocks() as demo:
  with gr.Column():

        gr.Markdown(
        """
       # Einzugshilfe (für Studenten)
        ## Beschreibung: Diese Anwendung gibt dir einen Einblick, wo du unter Berücksichtigung deiner Bedürfnisse und Wünsche in welches Stadteil du einziehen kannst.
        Es wird mithilfe der Kosinus-Ähnlichkeit drei Vorschläge getätitgt, wo die Werte der Merkmale berücksichtigt werden.
        Folgende Eingabedaten-Merkmale gibt es:
        - Fahrradweg-Infrastruktur
        - Anzahl von Fahrradhäuschen
        - Kaltmiete
        - Anzahl der Wohnungen
        - Durchschnitt der Relation Anzahl der Wohnungen und Größe des Bezirks
        - Daten zur Luftqualität
        - Anzahl der Grünflächen in den Bezirken


        Ausgabe:

        Vorschläge:
        - Top 3 Stadtteile, die du für den Einzug in Betracht ziehen solltest
        """
        )

        gr.Markdown(
        """
        ## Welche Merkmale sind für dich wichtig?
        ## Sobald diese gewählt sind, drücke auf `Start`
        """
        )

        with gr.Column():

            in_infra_rad = gr.Textbox(label="Anzahl Punkte (Rad) in Stadtteil", placeholder="Auswahl zwischen 0 und 20497", value="0")

            in_miet = gr.Textbox(label="Gewünschter Mietpreis", placeholder="Auswahl zwischen 0.0 und 14.8170392057951", value="0.0" )

            in_gruen = gr.Textbox(label="Gruenfläche", placeholder="Auswahl zwischen 0 und 446.021", value="0.0")

            in_aqi = gr.Textbox(label="AQI", placeholder="Auswahl zwischen 1.9174695298897275 und 2.211251302234055", value="1.9174695298897275")

            in_rad_house = gr.Textbox(label="Fahrradhausanzahl", placeholder="Auswahl zwischen 0 und 152", value="0")

            in_flach = gr.Textbox(label="Flaeche in km²", placeholder="Auswahl zwischen 0.5489340210934905 und35.36257777026979", value="0.5489340210934905")

            out = gr.Textbox(label=" Top 3 Stadtteile für deinen Einzug")


  with gr.Column():
    start_btn_single = gr.Button("Start")
    start_btn_single.click(fn=make_recommandation, inputs=[in_infra_rad, in_miet, in_gruen, in_aqi, in_rad_house, in_flach], outputs=[out])


demo.launch(debug=True)
